In [11]:
%pip install certifi
%pip install pyarrow

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

     ---------------------------------------- 25.1/25.1 MB 9.8 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import certifi
os.getcwd()

'c:\\Users\\Pranav Belmannu\\Documents\\GitHub\\RTA2024test\\MachineLearning'

In [4]:
header_data=pd.read_parquet('header_data.parquet',"pyarrow")
urls=pd.read_parquet('urls.parquet',"pyarrow")

In [5]:
#header_data.head()
urls.head(10)

,url
0,almaleasdancestudio.com
1,reddeerspecialties.com
2,nopopaws.com
3,burlingamedentalarts.com
4,sachsfamilydental.com
5,bhgidoc.com
6,billcollectorcharter.com
7,imperialtrainingcenter.com
8,physiocan.com
9,1stle.com


In [7]:
def extract_company_name(soup):
    
    if soup.find(id,'SITE_FOOTER'):
        text = soup.find(id,'SITE_FOOTER').get_text(strip=True)
        return text.split('\n')[0]
    
    for potential_name in ['company', 'business', 'organization', 'name']:
        meta = soup.find('meta', attrs={'name': potential_name})
        if meta and meta.get('content'):
            return meta.get('content')
    
    return None

In [8]:
https = 'https://'
http = 'http://'
no_company_names=[]
company_names=[]

for index, row in urls.iterrows():
    url = row['url']
    
    # Try HTTPS first
    full_url = https + url if not url.startswith('http') else url
    
    try:
        response = requests.get(full_url, verify=False)
        response.raise_for_status()  # Raise an exception for HTTP errors
    except requests.exceptions.RequestException as e:
        #print(f"HTTPS failed for {full_url}: {e}")
        
        full_url = http + url if not url.startswith('http') else url.replace('https://', 'http://')
        try:
            response = requests.get(full_url, verify=False)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            #print(f"HTTP also failed for {full_url}: {e}")
            continue  

    soup = BeautifulSoup(response.content, 'html.parser')
    
    
    if extract_company_name(soup):
        company_names.append({'url': full_url, 'company_name': extract_company_name(soup)})
    else:
        print(f"Company Name not found for {full_url}")
        no_company_names.append(full_url)

# Print or save the results
print("Company Names Extracted:")
for item in company_names:
    print(f"URL: {item['url']} - Company Name: {item['company_name']}")

print("\nURLs with No Company Name Found:")
for url in no_company_names:
    print(url)
    


c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'almaleasdancestudio.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.almaleasdancestudio.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'reddeerspecialties.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\ProgramData\Anaconda3\lib\si

Company Name not found for http://nopopaws.com


c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'burlingamedentalarts.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.burlingamedentalarts.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Company Name not found for https://burlingamedentalarts.com


c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'myoremdentist.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

Company Name not found for https://btipartners.com


c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ruppertnurseries.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Company Name not found for https://ruppertnurseries.com


c:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'fredhaulinginc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Company Name not found for https://fredhaulinginc.com
